In [112]:
pip install colorama

In [113]:

import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [114]:

with open('/content/drive/MyDrive/ChatBots/intents.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [115]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [116]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [117]:

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 20, 16)            16000     
_________________________________________________________________
global_average_pooling1d_8 ( (None, 16)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_25 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_26 (Dense)             (None, 21)                357       
Total params: 16,901
Trainable params: 16,901
Non-trainable params: 0
_________________________________________________________________


In [118]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500
3/3 [==============================] - 1s 13ms/step - loss: 3.0452 - accuracy: 0.0267
Epoch 2/500
3/3 [==============================] - 0s 14ms/step - loss: 3.0441 - accuracy: 0.0533
Epoch 3/500
3/3 [==============================] - 0s 13ms/step - loss: 3.0433 - accuracy: 0.1200
Epoch 4/500
3/3 [==============================] - 0s 17ms/step - loss: 3.0427 - accuracy: 0.0933
Epoch 5/500
3/3 [==============================] - 0s 16ms/step - loss: 3.0421 - accuracy: 0.0933
Epoch 6/500
3/3 [==============================] - 0s 11ms/step - loss: 3.0415 - accuracy: 0.0933
Epoch 7/500
3/3 [==============================] - 0s 13ms/step - loss: 3.0408 - accuracy: 0.0933
Epoch 8/500
3/3 [==============================] - 0s 15ms/step - loss: 3.0402 - accuracy: 0.0933
Epoch 9/500
3/3 [==============================] - 0s 11ms/step - loss: 3.0395 - accuracy: 0.0933
Epoch 10/500
3/3 [==============================] - 0s 12ms/step - loss: 3.0388 - accuracy: 0.0933
Epoch 11/500
3/3 [=

In [119]:
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model/assets


In [120]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("/content/drive/MyDrive/ChatBots/intents.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: hi
ChatBot: Hello
User: how are you?
ChatBot: I'm good, Thank you!
User: who are you?
ChatBot: I.m Thala, your bot assistant
User: can you help me
ChatBot: Tell me how can assist you
User: what are the courses you offer
ChatBot: Machine Learing, Data Science, Scientific Python, ISS Coaacing Program
User: can you provide me information about machine learning course
ChatBot: Key Benifits:Learn Statistics and ML Concepts from Academic Experts,Real-world case studies to build industry context,Hands-on projects to enhance coding skills,Major Project to consolidate your learning,Program Structure: 3 Months Program,,100+ Hrs Virtual Classes,10+ Case Studies,5+ Webinars by Industry Experts,1 Major Project
User: fee of machine learning course
ChatBot: 20000+GST
User: Data science
ChatBot: Checkout link : https://en.wikipedia.org/wiki/Data_science
User: contact details
ChatBot: ok
User: how can i  contact you
ChatBot: visit : https://thalam